In [1]:
import pinecone      
import os
import json

from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings
import pinecone
from langchain.document_loaders import JSONLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

C:\Users\nauman.tasawar\AppData\Roaming\Python\Python311\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
openai_api_key = os.environ["THINK_MALL_GPT_KEY"]
PINECONE_API_KEY = os.environ["PINE_API_KEY"]

PINECONE_API_ENV = os.environ["PINE_ENV"]

In [3]:
pinecone.init(
    api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV
)

In [16]:
file_path = 'entertainments.json'
with open(file_path, 'r') as file:
    json_data = json.load(file)

In [17]:
def generate_description(shop_name, shop_data):
    description = shop_data.get('Description', '')
    working_hours = shop_data.get('Working Hours', '')
    contact_number = shop_data.get('Contact Number', '')
    location = shop_data.get('Location', '')

    similar_brands = shop_data.get('Similar Brands', '')
    other_info = shop_data.get('Other Info', '')

    # Create a generic description sentence
    generic_description = f"{shop_name.capitalize()} is located in {location}. "
    if description:
        generic_description += f"{description}. "

    if working_hours:
        generic_description += f"We are open {working_hours}. "

    if contact_number:
        generic_description += f"You can reach us at {contact_number}. "

    if similar_brands:
        generic_description += f"We are similar to brands like {similar_brands}. "

    if other_info:
        generic_description += f"Additional information: {other_info}"

    return generic_description.strip()



In [18]:
# Assuming 'data' contains your JSON structure

# Lists to store information
headings = []
descriptions = []
docs = []

for category, shops in json_data.items():
    for shop_name, shop_data in shops.items():
        description = generate_description(shop_name, shop_data)
        location = shop_data.get('Location', 'Unknown Location')
        
        # Create the heading
        heading = f"{category} - {shop_name.capitalize()} - {location.capitalize()}"
        headings.append(heading)
        
        # Create the description
        descriptions.append(description)
        
        docs.append(Document(
        page_content=description,
        metadata={"category": category, "shop name": shop_name, "location": location.capitalize()}))
    

# Print the information in separate lists

# for heading , description in (zip(headings,descriptions)):
#     print(heading)
#     print(description)
#     print("\n")

# print("\nDescriptions:")
# for description in descriptions:
#     print(description)


In [19]:
docs

[Document(page_content='Fun city is located in Andalusia Court. Fun City is an ideal play centre for kids between 1-12 years of age. Fun City forms an integral part in the society by providing a platform to children for social interactions and active play in a safe, secure and conveniently accessed location.\nFun has many faces at Fun City. “Fun n Learn” an area for children age 0-4 provides avenues for the mental and physical development of a child, whereas the softplay area called “Play Zone” provides a safe and secure place to slide, run, climb, jump around and enjoy. Birthday parties, School Trips and Mom’s Coffee Mornings are the perfect get-togethers that make every moment for family and friends a whole lot more fun and memorable.\nFun City has a simple invitation to one and all; Come, play! . We are open Open at 10:00 AM. You can reach us at 043669819. We are similar to brands like nan. Additional information: www.funcity.ae\nWeekday (Sunday to Wednesday)            : 2 PM to 10

In [20]:
embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002",openai_api_key=openai_api_key) 

In [21]:
vectorstore = Pinecone.from_documents(
    docs, embeddings, index_name="think-mall"
)

In [75]:
text = "what is your name"
text_2 = " list only engineers"

query_vector = embeddings.embed_query(text_2)
query_vector2 = embeddings.embed_query(text_2)

In [76]:
result = index.query(
  vector=query_vector,
  top_k=5,
  include_values=True,
  include_metadata=True
)

In [77]:
matches = result['matches']

for match in matches:
    print(f"ID: {match['id']}, Score: {match['score']}")

ID: SERVICES - Technology people - Andalusia court, Score: 0.725338042
ID: ELECTRONICS / TELECOMMUNICATIONS - Eros - Persia court, Score: 0.722348332
ID: SERVICES - T-shirt factory / portfolio - Egypt court, Score: 0.721653879
ID: ELECTRONICS / TELECOMMUNICATIONS - Technology people - Andalusia court, Score: 0.720507801
ID: SERVICES - Sell your motors - Tunisia court, Score: 0.720393836


In [66]:
matches

[{'id': 'COSMETICS / PERFUMES - Aroma gallery - Tunisia court',
  'metadata': {},
  'score': 0.887777507,
  'values': [0.00573337078,
             0.00548007572,
             0.0124251563,
             -0.0224542767,
             -0.0110012265,
             0.0212494135,
             -0.0251652189,
             -0.0089474814,
             0.0113024423,
             -0.0120144067,
             0.0286155101,
             -0.00270580873,
             -0.000867707189,
             -0.0267397575,
             -0.0133835701,
             -0.00204861048,
             0.0365566574,
             0.00398768764,
             0.000332449941,
             -0.0048571066,
             0.0031610555,
             0.00393292122,
             -0.0196338,
             -2.67815813e-05,
             0.0227828752,
             -0.00895432755,
             0.0177991223,
             -0.0492351092,
             -0.00112442533,
             -0.0113092875,
             0.000784701668,
             0.00188602228,

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
llm.predict("Hello world!")

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory(memory_key="chat_history", return_messages= True)
chain = ConversationalRetrievalChain.from_llm(llm, retriever= retriever, memory= memory)

In [ ]:
vector_db = Pinecone.from_existing_index(index_name, embedding_function)